<a href="https://colab.research.google.com/github/hemanth346/eva4/blob/master/S15A/Dataset2_nocompression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [1]:
!nvidia-smi

Sun May 10 07:36:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import cv2
import matplotlib.pyplot as plt

In [3]:
! ls '/content/drive/My Drive/DL/DepthModel/Dataset'

annotations.txt  depth_maps  fg_bg  fg_bg_masks  inp_bg  inp_fg  inp_fg_masks


In [0]:
data_dir = os.path.join('/content/drive/My Drive/DL/DepthModel', 'Dataset')
# Images scrapped from internet 
# required inputs
# fg = Foreground
# bg - background
bg_dir = os.path.join(data_dir, 'inp_bg')
fg_dir = os.path.join(data_dir, 'inp_fg')
# can be generated using alpha channel thresholding of fg images
mask_dir = os.path.join(data_dir, 'inp_fg_masks')
# generated using given inputs
fg_bg_dir = os.path.join(data_dir, 'fg_bg')
fg_bg_mask_dir = os.path.join(data_dir, 'fg_bg_masks')
depth_dir = os.path.join(data_dir, 'depth_maps')

# others
annot_file = os.path.join(data_dir, 'annotations.txt')
log_file = os.path.join('/content/drive/My Drive/DL/DepthModel', 'timer.log')

# generate archive parallelly
compressed_dir = os.path.join('/content/drive/My Drive/DL/DepthModel', 'Compressed_Dataset')
compressed_fg_bg = os.path.join(compressed_dir, 'fg_bg')
compressed_fg_bg_mask = os.path.join(compressed_dir, 'fg_bg_masks')
compressed_depth_dir = os.path.join(compressed_dir, 'depth_maps')

In [0]:
from pathlib import Path

Path(fg_bg_dir).mkdir(exist_ok=True)
Path(fg_bg_mask_dir).mkdir(exist_ok=True)
Path(depth_dir).mkdir(exist_ok=True)

Path(compressed_fg_bg).mkdir(parents=True, exist_ok=True)
Path(compressed_fg_bg_mask).mkdir(parents=True, exist_ok=True)
Path(compressed_depth_dir).mkdir(parents=True, exist_ok=True)


In [6]:
if log_file:
  print('Logging...')
  import logging
  logger = logging.getLogger()
  log = logging.FileHandler(log_file)
  log.setLevel(logging.DEBUG)
  template = logging.Formatter('%(asctime)s : %(message)s')
  log.setFormatter(template)
  logger.addHandler(log)

Logging...


In [0]:
# adding Depth maps

In [8]:
! git clone https://github.com/hemanth346/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 244 (delta 4), reused 7 (delta 2), pack-reused 235
Receiving objects: 100% (244/244), 11.80 MiB | 39.23 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [9]:
cd DenseDepth/

/content/DenseDepth


In [10]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O nyu.h5

--2020-05-10 07:37:03--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.40.75
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.40.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘nyu.h5’

nyu.h5              100%[===================>] 164.89M  20.3MB/s    in 9.0s    

2020-05-10 07:37:13 (18.4 MB/s) - ‘nyu.h5’ saved [172897376/172897376]



In [11]:
%tensorflow_version 1.x
import os
import glob
from keras.models import load_model
from matplotlib import pyplot as plt

from layers import BilinearUpSampling2D
from utils import predict_single_file
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

# Load model into GPU / CPU
model = load_model('nyu.h5', custom_objects=custom_objects, compile=False)

In [0]:
# random overlay on Bg

In [0]:
import os
import cv2
import random
import matplotlib.pyplot as plt
import timeit
import datetime
def overlay(bg, fg, mask, number=20, save=False, name='filename', 
            fg_bg_dir=fg_bg_dir, fg_bg_mask_dir=fg_bg_mask_dir, 
            bbox_file=None, depth_dir=None, show=False):
    """
    Given a Background and Foreground images with alphachannel(flag -1 in cv2.imread)
    place foregroung on top of background randomly given number of times
    
    Also place corresponding mask on a black canvas, to match the fg placement
    """
    if fg[:, :, 3] is None:
        print('No alpha channel detected in FG image')
        return 1
    
    # To stop fg from overshooting bg canvas
    x_offset_limit = bg.shape[1] - fg.shape[1]
    y_offset_limit = bg.shape[0] - fg.shape[0]

    
    # width - X axis starting points from left
    x_offsets = random.sample(range(1, x_offset_limit), 20)

    # height - Y axis starting points from top
    y_offsets = random.sample(range(1, y_offset_limit), 20)
    
    for num, (x_offset, y_offset) in enumerate(zip(x_offsets, y_offsets)):
        # extracting only file name irrespective of extension, if given
        fname = os.path.splitext(name)[0]+'_'+str(num+1)

        # BBOX coord - (x1, y1) and (x2, y2)
        y1, y2 = y_offset, y_offset + fg.shape[0]
        x1, x2 = x_offset, x_offset + fg.shape[1]

        alpha_fg = fg[:, :, 3] / 255.0
        alpha_bg = 1.0 - alpha_fg
        
        # Not disturbing orignal images
        fg_bg = bg.copy()
        black = bg * np.zeros(bg.shape, dtype=np.int8)
        
        for c in range(0, 3):
            fg_bg[y1:y2, x1:x2, c] = (alpha_fg * fg[:, :, c] +
                                      alpha_bg * fg_bg[y1:y2, x1:x2, c])
            # if mask not None:
            black[y1:y2, x1:x2, c] = (alpha_fg * mask[:, :, c] +
                                          alpha_bg * black[y1:y2, x1:x2, c])
        # convert mask to single channel grayscale
        black = black[:,:,0]
        # cv2.cvtColor(black, cv2.COLOR_BGR2GRAY)
        # # if mask not None:
        # roi = black[y1:y2, x1:x2]
        # print('Shapes : ',roi.shape, mask.shape)
        # fg_mask_roi = cv2.add(roi,mask)
        # black[y1:y2, x1:x2] = fg_mask_roi

        if save:
            # os.mkdir(fg_bg_dir) if not os.path.exists(fg_bg_dir)
            if not os.path.exists(fg_bg_dir):
                os.mkdir(fg_bg_dir) 
            fg_bg_fname = os.path.join(fg_bg_dir, fname+'.jpg')
            # set quality of JPEG to 50. default 95, lowest 0
            cv2.imwrite(fg_bg_fname, fg_bg, [cv2.IMWRITE_JPEG_QUALITY, 50])
            # if mask:
            # os.mkdir(fg_bg_mask_dir) if not os.path.exists(fg_bg_mask_dir)
            if not os.path.exists(fg_bg_mask_dir):
                os.mkdir(fg_bg_mask_dir)
            mask_fname = os.path.join(fg_bg_mask_dir, fname+'.jpg')
            # set quality of JPEG to 30. default 95, lowest 0
            cv2.imwrite(mask_fname, black, [cv2.IMWRITE_JPEG_QUALITY, 30])
            # print('FG_BG shape, FG_BG_mask_shape : ', fg_bg.shape, black.shape)
            if bbox_file:
                # each line will have corresponding bbox
                with open(bbox_file, 'w+') as f:
                    coord = ', '.join(list(map(str, (x1, y1, x2, y2))))
                    f.write(coord+' '+fname)
            if depth_dir:
              depth_img = predict_single_file(fg_bg_fname,model=model)
              if not os.path.exists(depth_dir):
                  os.mkdir(depth_dir)
              depth_fname = os.path.join(depth_dir, fname+'.jpg')
              # set quality of JPEG to 30. default 95, lowest 0
              cv2.imwrite(depth_fname, depth_img, [cv2.IMWRITE_JPEG_QUALITY, 90])


        if show:
            print(f'bbox x1,y1 and x2,y2: ', x1, y1, x2, y2)
            fig, ax = plt.subplots(1, 2, sharex=True, sharey=True)
            axes = ax.flatten()
            axes[0].imshow(fg_bg)
            axes[1].imshow(black)
            plt.show()

In [15]:
import os
import numpy as np
import datetime
import shutil
from tqdm.notebook import tqdm

annot_file = os.path.join(data_dir, 'annotations.txt')
# starting from 24 bg as others are already written to drive 
# in previous after 23 bg, content not written to drive
# add 2
for idx, fname in enumerate(os.listdir(bg_dir)[25:101]):
    bg_start = datetime.datetime.now()
    abspath = os.path.join(bg_dir, fname)
    bg = cv2.imread(abspath)
    logger.debug('BG Original shape : {0}'.format(bg.shape))
    bg = cv2.resize(bg, (224, 224), interpolation=cv2.INTER_CUBIC)
    logger.debug('Resized BG shape {0}'.format(bg.shape))
    bg_fname = os.path.splitext(fname)[0]
    current_fg_bg_dir = os.path.join(fg_bg_dir, bg_fname)
    current_fg_bg_mask_dir = os.path.join(fg_bg_mask_dir, bg_fname)
    current_depth_dir = os.path.join(depth_dir, bg_fname)
    # create a separate directory for each bg
    Path(current_fg_bg_dir).mkdir(parents=True, exist_ok=True)
    Path(current_fg_bg_mask_dir).mkdir(parents=True, exist_ok=True)
    Path(current_depth_dir).mkdir(parents=True, exist_ok=True)
    for fname in tqdm(os.listdir(fg_dir)):
        # print('File name : ', fname)
        fg_img = os.path.join(fg_dir, fname)
        fg_mask = os.path.join(mask_dir, fname)
        # set flag to -1 to read alpha channels as well
        fg = cv2.imread(fg_img, -1)
        mask = cv2.imread(fg_mask, -1)
        logger.debug('FG, Mask Original shape : {0},{1}'.format(fg.shape, mask.shape))
        # could've sent this inside overlay and resize mask randomly as well 20 times
        fg = cv2.resize(fg, (100, 70), interpolation=cv2.INTER_AREA)
        mask = cv2.resize(mask, (100, 70), interpolation=cv2.INTER_AREA)
        # print('FG, Mask resized shape',fg.shape, mask.shape)
        start = datetime.datetime.now()
        # print('Time for resizes : {0}'.format(start - bg_start))        
        overlay(bg=bg, fg=fg, mask=mask,
                name=bg_fname+'__'+fname,  
                fg_bg_dir=current_fg_bg_dir, 
                fg_bg_mask_dir=current_fg_bg_mask_dir,
                depth_dir=current_depth_dir, bbox_file=annot_file,
                number=20, save=True, show=False)
        # flip images and overlay them
        # https://note.nkmk.me/en/python-opencv-numpy-rotate-flip/
        logger.info('Time for overlay before flip: {0}'.format(
            datetime.datetime.now() - start
            ))
        start = datetime.datetime.now()
        fg_flip = cv2.flip(fg, flipCode=1)
        mask_flip = cv2.flip(mask, flipCode=1)
        # Added additional names at the beginning
        # at the end it will be removed while spliting file extension
        overlay(bg=bg, fg=fg_flip, mask=mask_flip,
                name=bg_fname+'__flip_'+fname, 
                fg_bg_dir=current_fg_bg_dir, 
                fg_bg_mask_dir=current_fg_bg_mask_dir,
                depth_dir=current_depth_dir, bbox_file=annot_file,
                number=20, save=True, show=False)
        logger.info('Time for overlay after flip: {0}'.format(
            datetime.datetime.now() - start
            ))
        # print('-'*27)
        # break
    # print('Archiving... ')
    # start = datetime.datetime.now()
    # shutil.make_archive(os.path.join(compressed_fg_bg, bg_fname), 'zip', current_fg_bg_dir)
    # shutil.make_archive(os.path.join(compressed_fg_bg_mask, bg_fname), 'zip', current_fg_bg_mask_dir)
    # shutil.make_archive(os.path.join(compressed_depth_dir, bg_fname), 'zip', current_depth_dir)
    # logger.info('Time for archiving {0}'.format(
    #     datetime.datetime.now() - start
    #     ))
    logger.debug('*'*27)
    # archive breaking write operations to drive
    # flush unmount and force remount taking too much time
    # and mostly resulting in crash
    # if idx+1 % 10 == 0:
    #   # To avoid data loss
    #   print('Flush Umount and Force remount')
    #   drive.flush_and_unmount()
    #   drive.mount('drive', force_remount=True)

    logger.debug('Time for BG {0} : {1}'.format(
        abspath,
        datetime.datetime.now() - bg_start
        ))
    print('*'*27)
    print('Time for BG {0} : {1}'.format(
        abspath,
        datetime.datetime.now() - bg_start
        ))


***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office26.jpeg : 0:06:43.083637



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office27.jpeg : 0:03:59.895742



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office28.jpeg : 0:03:56.122787



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office29.jpeg : 0:03:58.524599



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office30.jpeg : 0:03:56.820358


...


***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office97.jpeg : 0:04:08.157869



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office98.jpeg : 0:04:07.699264



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office99.jpeg : 0:04:12.941703



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office100.jpeg : 0:04:08.977222



***************************
Time for BG /content/drive/My Drive/DL/DepthModel/Dataset/inp_bg/office101.jpeg : 0:04:06.106699


In [0]:
# files not syncing with google drive.. Flush unmount seems to work.. 
# Ref : https://github.com/googlecolab/colabtools/issues/287#issuecomment-517446876
# for remount also seems to work if work still in progress
# https://github.com/googlecolab/colabtools/issues/287#issuecomment-606528171
# modifying Drive folders on the GUI while having it mount seems to cause the bug??
# Ref : https://github.com/googlecolab/colabtools/issues/287#issuecomment-514478749